# Importing Libraries

In [311]:
import requests
import pandas as pd
from tqdm.auto import tqdm
import sqlalchemy as db
from fuzzywuzzy import fuzz

# Creating the connection with database

Transforming the table into a pandas Dataframe

In [312]:
engine = db.create_engine(f'{DATABASE_URL}')
conn = engine.connect()
df_ingredientes = pd.read_sql('ingredient', con=conn)

# Search for recipes

## User input (ingredients)

In [349]:
user_ingredients = []
while True:
    ingredient = str(input('Ingredientes:'))
    if ingredient == 'ok':
        break
    else:
        user_ingredients.append(ingredient)

Ingredientes:ovo
Ingredientes:chocolate
Ingredientes:farinha
Ingredientes:manteiga
Ingredientes:ok


In [350]:
user_ingredients

['ovo', 'chocolate', 'farinha', 'manteiga']

## Returning the ingredient's ids

In [352]:
id_ingredient_user = []
for ingredient in user_ingredients:
    lst_id_ingredient = df_ingredientes.loc[df_ingredientes['ingrediente'] == f'{ingredient}', :]['id_ingrediente'].values
    
    if len(lst_id_ingredient) > 0:
        id_ingredient = lst_id_ingredient[0]
        id_ingredient_user.append(id_ingredient)
    else:
        for unique_ingredient in df_ingredientes['ingrediente']:
                ratio = fuzz.token_set_ratio(ingredient, unique_ingredient)
                if ratio > 85:
                    lst_id_ingredient = df_ingredientes.loc[df_ingredientes['ingrediente'] == f'{unique_ingredient}', :]['id_ingrediente'].values
                    id_ingredient = lst_id_ingredient[0]
                    id_ingredient_user.append(id_ingredient)
                    break

In [353]:
id_ingredient_user = tuple(id_ingredient_user)

## Returning the recipes

In [399]:
engine = db.create_engine(f'{DATABASE_URL}')
conn = engine.connect()
query = f'''SELECT 
                        t2.href,
                        t2.rendimento,
                        t2.tempo_preparo,
                        t2.nome,
                        t2.qtde_ingrediente_usuario,
                        t2.qtd_ingrediente,
                        t2.ingredients_ratio,
                        t2.categoria,
                        rim.image_url
                    FROM
                        (SELECT 
                            *
                        FROM
                            (SELECT 
                            r.href,
                                r.rendimento,
                                r.tempo_preparo,
                                r.nome,
                                COUNT(i.id_ingrediente) AS qtde_ingrediente_usuario,
                                r.qtd_ingrediente,
                                (COUNT(i.id_ingrediente) / r.qtd_ingrediente) AS ingredients_ratio,
                                r.categoria
                        FROM
                            ingredient i
                        JOIN recipe_ingredient ri ON i.id_ingrediente = ri.id_ingredient
                        JOIN recipes_df r ON r.id_recipes_df = ri.id_recipe
                        WHERE
                            i.id_ingrediente IN {id_ingredient_user}
                        GROUP BY ri.id_recipe
                        ORDER BY qtde_ingrediente_usuario DESC) AS t
                        WHERE
                            t.ingredients_ratio > 0.6
                            LIMIT 200) AS t2
                            LEFT JOIN
                        recipes.images rim ON t2.href = rim.href
                        WHERE rim.image_url IS NOT NULL
                        ORDER BY t2.ingredients_ratio DESC
                        LIMIT 21'''

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\julia\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 693, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\julia\anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\julia\anaconda3\lib\site-packages\sqlalchemy\dialects\mysql\base.py", line 2302, in do_rollback
    dbapi_connection.rollback()
  File "C:\Users\julia\anaconda3\lib\site-packages\mysql\connector\connection_cext.py", line 386, in rollback
    self._cmysql.rollback()
_mysql_connector.MySQLInterfaceError: Lost connection to MySQL server during query


In [400]:
results = engine.execute(query)
recipes = [recipe for recipe in results]

In [394]:
recipes[:5]

[('https://www.tudogostoso.com.br/receita/164057-bolo-recheado.html', '10 porções', '30MIN', 'Bolo recheado', 4, 10, Decimal('0.4000')),
 ('https://www.tudogostoso.com.br/receita/154123-melhor-bolo-prestigio.html', '15 porções', '150MIN', 'Melhor bolo prestígio', 4, 11, Decimal('0.3636')),
 ('https://www.tudogostoso.com.br/receita/129074-muffin-com-gotas-de-chocolate.html', '22 porções', '30MIN', 'Muffin com gotas de chocolate', 4, 9, Decimal('0.4444')),
 ('https://www.tudogostoso.com.br/receita/47472-bolo-tipo-prestigio.html', '18 porções', '60MIN', 'Bolo tipo prestígio', 4, 9, Decimal('0.4444')),
 ('https://www.tudogostoso.com.br/receita/123100-cookie.html', '30 porções', '30MIN', 'Cookie', 4, 8, Decimal('0.5000'))]